In [1]:
from config import  functions
from dotenv import load_dotenv

import os

load_dotenv()

True

In [2]:
import numpy as np 
import random 
def get_function_params(function_detail):
    function_name = function_detail["name"]
    function_parameters = function_detail["parameters"]["properties"]
    required_parameters = function_detail["parameters"].get("required", [])
    optional_parameters = function_detail["parameters"].get("optional", [])
    
    function_calls = []
    
    function_args_choices_req = {}
    
    for param in required_parameters:
        param_details = function_parameters[param]
        param_type = param_details["type"]
        if 'enum' in param_details:
            function_args_choices_req[param] = param_details['enum']
        else:
            if param_type == "boolean":
                function_args_choices_req[param] = [True, False]
            elif param_type in ["integer", "number"]:
                lower_bound = param_details.get("lower_bound", 1)
                upper_bound = param_details.get("upper_bound", 100)
                print(function_name, lower_bound, upper_bound,param_details)
                function_args_choices_req[param] = list(range(lower_bound, upper_bound))
            elif param_type == "string":
                function_args_choices_req[param] = ["|string1|", "|string2|", "|string3|"]
        
    function_args_choices_opt = {}
    for param in optional_parameters:
        param_details = function_parameters[param]
        param_type = param_details["type"]
        if 'enum' in param_details:
            function_args_choices_opt[param] = param_details['enum']
        else:
            if param_type == "boolean":
                function_args_choices_opt[param] = [True, False]
            elif param_type in ["integer", "number"]:
                lower_bound = param_details.get("lower_bound", 1)
                upper_bound = param_details.get("upper_bound", 100)
                print(function_name, lower_bound, upper_bound,param_details)
                function_args_choices_opt[param] = list(range(lower_bound, upper_bound))
            elif param_type == "string":
                function_args_choices_req[param] = ["|string1|", "|string2|", "|string3|"]

    return function_name, function_args_choices_req, function_args_choices_opt

In [6]:
def construct_function(required_params_config, opt_params_config, opt_prob=0.5):
    function_args = {}
    for param, choices in required_params_config.items():
        function_args[param] = random.choice(choices)
    
    for param, choices in opt_params_config.items():
        if random.random() < opt_prob:
            function_args[param] = random.choice(choices)
    
    return function_args


def generate_function_call_args(function_details, n=10, opt_prob=0.5, function_args={}):
    
    for function_detail in function_details:
        function_name, required_params_config, opt_params_config = get_function_params(function_detail)
        function_args[function_name] = []
        
        for _ in range(n):
            args = construct_function(required_params_config, opt_params_config, opt_prob=opt_prob)
            function_args[function_name].append(args)
    
    return function_args


calling_params = generate_function_call_args(functions, n=50) ## Generate 10 function calls for each function

## Assimilate all function calls into a readable and maintainable format

function_call_main_dict = {}
for function_name, function_args in calling_params.items():
    function_call_main_dict[function_name] = {"args": [], "calls" : []}
    function_call_main_dict[function_name]['args'].append(function_args)
    for args in function_args:
        function_call = function_name + "("
        for param, value in args.items():
            function_call += f"{param}={value}, "
        function_call = function_call[:-2] + ")" if len(args) > 0 else function_call + ")"
        function_call_main_dict[function_name]['calls'].append(function_call)


adjust_temperature 1 80 {'type': 'number', 'description': 'The target temperature for the specified zone in degrees Celsius.', 'lower_bound': 1, 'upper_bound': 80}
activate_defroster 1 30 {'type': 'integer', 'description': 'Duration in minutes for which the defroster should be active.', 'default': 10, 'lower_bound': 1, 'upper_bound': 30}
play_music 1 10 {'type': 'integer', 'description': 'Volume level from 1 (low) to 10 (high).', 'default': 5, 'lower_bound': 1, 'upper_bound': 10}
control_ambient_lighting 1 10 {'type': 'integer', 'description': 'The intensity level of the lighting, from 1 (low) to 10 (high).', 'default': 5, 'lower_bound': 1, 'upper_bound': 10}
set_cruise_control 10 150 {'type': 'integer', 'description': 'The cruise control speed in km/h.', 'lower_bound': 10, 'upper_bound': 150}


In [7]:
import json 
with open("./data/function_calls.json", "w") as f:
    json.dump(function_call_main_dict, f)

In [8]:
# read the saved function calls
with open("./data/function_calls.json", "r") as f:
    function_calls = json.load(f)